# Hindsight-OpenAI Tutorial

**A drop-in replacement for the OpenAI Python client with automatic memory integration**

## What is Hindsight-OpenAI?

`hindsight-openai` is a transparent wrapper around the official OpenAI Python client that automatically:

- 🧠 **Injects relevant memories** from your Hindsight system into conversations
- 💾 **Stores conversation history** to Hindsight for future retrieval  
- 🔄 **Works seamlessly** with existing OpenAI code (just change the import)
- ⚡ **Supports both sync and async** clients

## Why Use It?

### The Problem

AI assistants typically have no memory of previous conversations. Each interaction starts fresh, requiring you to:
- Repeat context manually
- Copy-paste relevant information
- Build custom RAG pipelines
- Manage conversation history yourself

### The Solution

Hindsight-OpenAI gives your AI **automatic long-term memory**:
- Remembers past conversations
- Recalls user preferences and facts
- Maintains context across sessions
- Zero code changes to your existing OpenAI usage

## Prerequisites

1. **Hindsight API server running** (see main Hindsight README)
2. **OpenAI API key** or compatible API (Groq, OpenRouter, etc.)
3. **Python >= 3.10**

## Setup

First, let's set up our environment and configure Hindsight integration:

In [ ]:
import os
from hindsight_openai import configure, AsyncOpenAI

# Set your API keys
# Option 1: Use Groq (fast and free)
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "your-groq-api-key")
if not GROQ_API_KEY:
    raise ("GROQ_API_KEY not set") 

# Option 2: Use OpenAI
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "sk-...")

# Configure Hindsight integration
configure(
    hindsight_api_url="http://localhost:8888",  # Your Hindsight API server
    agent_id="tutorial-user",                   # Unique ID for this user/agent
    store_conversations=True,                   # Auto-save conversations
    inject_memories=True,                       # Auto-inject relevant context
)

print("✓ Hindsight configured successfully!")
print("")
print("NOTE: This tutorial uses AsyncOpenAI which works perfectly in Jupyter notebooks.")
print("For regular Python scripts, you can use the sync OpenAI client instead.")

## Example 1: Basic Usage (No Changes Needed!)

Use the OpenAI client exactly as you normally would. Hindsight works transparently in the background.

In [ ]:
# Create client (using Groq's OpenAI-compatible API)
client = AsyncOpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1",
)

# First conversation - establish some facts
print("=== First Conversation ===")
response = await client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "user", "content": "My name is Alice and I love Python programming!"}
    ],
)

print(f"Assistant: {response.choices[0].message.content}\n")
print("→ This conversation is now stored in Hindsight!")

## Example 2: Memory Injection in Action

Now ask a question that requires remembering the previous conversation:

In [ ]:
print("=== Second Conversation (with memory) ===")
response = await client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "user", "content": "What's my name and what do I like?"}
    ],
)

print(f"Assistant: {response.choices[0].message.content}\n")
print("→ Hindsight automatically injected relevant memories before this request!")
print("→ The AI knew your name and preferences without you repeating them.")

## How It Works

Behind the scenes, Hindsight-OpenAI:

### 1. **Memory Storage**
After each API call:
- Captures the full conversation context
- Stores it in Hindsight's semantic memory system
- Indexes it for fast retrieval

### 2. **Memory Injection**
Before each API call:
- Extracts the user's query
- Searches Hindsight for relevant past conversations
- Injects top memories as a system message

### What Gets Sent to OpenAI

Without Hindsight:
```python
messages = [
    {"role": "user", "content": "What's my name?"}
]
```

With Hindsight (automatic):
```python
messages = [
    {
        "role": "system",
        "content": "Relevant context from your memory:\n\n1. User's name is Alice\n   (Date: 2024-11-18)\n   (Type: world)"
    },
    {"role": "user", "content": "What's my name?"}
]
```

## Example 3: Multi-Turn Conversations

Build up context over multiple interactions:

In [ ]:
# Conversation 1: Share a preference
print("=== Conversation 1: Sharing preferences ===")
response = await client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "user", "content": "I'm working on a machine learning project using PyTorch."}
    ],
)
print(f"Assistant: {response.choices[0].message.content}\n")

# Conversation 2: Different topic
print("=== Conversation 2: Different topic ===")
response = await client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "user", "content": "I prefer functional programming over OOP."}
    ],
)
print(f"Assistant: {response.choices[0].message.content}\n")

# Conversation 3: Ask for recommendations
print("=== Conversation 3: Getting personalized advice ===")
response = await client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
        {"role": "user", "content": "Can you recommend a good book for me based on what you know?"}
    ],
)
print(f"Assistant: {response.choices[0].message.content}\n")
print("→ The AI used your programming interests and preferences to make recommendations!")

## Example 4: Document Grouping

Group related conversations using `document_id`:

In [ ]:
from hindsight_openai import configure

# Configure with document ID for a specific project
configure(
    hindsight_api_url="http://localhost:8888",
    agent_id="tutorial-user",
    document_id="ml-project-2024",  # All conversations tagged with this ID
)

# All these conversations will be grouped together
conversations = [
    "I'm using ResNet for image classification",
    "My dataset has 10,000 images",
    "Training accuracy is stuck at 65%",
]

for msg in conversations:
    response = await client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": msg}],
    )
    print(f"User: {msg}")
    print(f"Assistant: {response.choices[0].message.content}\n")

print("→ All these conversations are grouped under document 'ml-project-2024'")

## Example 5: Async Support

Works perfectly with AsyncOpenAI for high-throughput applications:

In [ ]:
from hindsight_openai import AsyncOpenAI
import asyncio

async def async_example():
    # Create async client
    async_client = AsyncOpenAI(
        api_key=GROQ_API_KEY,
        base_url="https://api.groq.com/openai/v1",
    )
    
    # Store a fact
    print("=== Storing fact ===")
    response = await async_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": "My favorite color is blue."}],
    )
    print(f"Assistant: {response.choices[0].message.content}\n")
    
    # Query with memory
    print("=== Querying with memory ===")
    response = await async_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": "What's my favorite color?"}],
    )
    print(f"Assistant: {response.choices[0].message.content}\n")

# Run async example
await async_example()
print("→ Async operations work seamlessly with Hindsight!")

## Configuration Options

Fine-tune Hindsight's behavior:

In [ ]:
from hindsight_openai import configure

# Full configuration example
configure(
    hindsight_api_url="http://localhost:8888",  # Hindsight API URL
    agent_id="my-agent",                        # Agent identifier (required)
    api_key=None,                               # Optional Hindsight API key
    
    # Features
    store_conversations=True,                   # Store conversations automatically
    inject_memories=True,                       # Inject memories automatically
    
    # Organization
    document_id="session-123",                  # Optional document grouping
    
    # Control
    enabled=True,                               # Master on/off switch
)

print("Configuration options explained:")
print("- store_conversations: Automatically save conversations to Hindsight")
print("- inject_memories: Automatically retrieve and inject relevant context")
print("- document_id: Group related conversations together")
print("- enabled=False: Disable Hindsight without changing code")

## Use Cases

### 1. **Personal AI Assistant**
- Remembers your preferences, work history, and interests
- Provides personalized recommendations
- Maintains context across days/weeks

### 2. **Customer Support Chatbot**
- Recalls previous support tickets
- Knows customer preferences and history
- Provides consistent, context-aware responses

### 3. **Research Assistant**
- Remembers documents you've discussed
- Connects related topics from different sessions
- Builds knowledge over time

### 4. **Code Review Tool**
- Remembers project architecture decisions
- Recalls past code review comments
- Maintains consistency across reviews

## Benefits Summary

✅ **Zero Code Changes** - Drop-in replacement for OpenAI client  
✅ **Automatic Context** - No manual RAG pipeline needed  
✅ **Long-term Memory** - Conversations persist across sessions  
✅ **Smart Retrieval** - Semantic search finds relevant context  
✅ **Both Sync/Async** - Works with any OpenAI client pattern  
✅ **Configurable** - Fine-tune behavior to your needs  
✅ **Transparent** - Original OpenAI responses unchanged  

## Next Steps

- **Explore Hindsight API**: Check out the main Hindsight README for advanced features
- **Customize Search**: Tune `memory_search_budget` for your use case
- **Use Document IDs**: Organize conversations by project/session
- **Try Different Models**: Works with OpenAI, Groq, Ollama, and more

## Resources

- [Hindsight Main README](../README.md) - Core memory system docs
- [Hindsight-OpenAI README](README.md) - Package documentation
- [OpenAI API Docs](https://platform.openai.com/docs/api-reference) - Original API reference